In [1]:
from nltk.corpus import sentiwordnet as swn
import json
import sys

In [36]:
# Deprecated. See get_score()
def get_scores(word, postag):
    """  Function to get sentiment scores from SentiWordNet.
    Args:
        word: word to get sentiment scores
        postag: pos tag of the word
    Returns:
        score: a senti_synset instance that has negative, positive and neutral
        scores of the word"""

    if postag == "Noun":  # NOUN
        score = swn.senti_synset(word + '.n.01')
        return score
    elif postag == "Verb":  # VERB
        score = swn.senti_synset(word + '.v.01')
        return score
    elif postag == "Adjective":  # ADJECTIVE
        score = swn.senti_synset(word + '.a.01')
        return score
    elif postag == "Adverb":  # ADVERB
        score = swn.senti_synset(word + '.r.01')
    return score

In [34]:
# Read Data
with open('/media/fatih/DATA/Grad/corrected_neg_eng.json') as data_file:
    data = data_file.readlines()

data = [x.strip() for x in data]
jsonSentences = [json.loads(x) for x in data]

In [35]:
# word = jsonSentences[0][0]["word"]
# pos_tag = jsonSentences[0][0]["posTag"]
# word = "bad"
# pos_tag = "Adjective"
# print("Word: " + word)
# print("Pos tag: " + pos_tag)
# score = get_score(word, pos_tag)
# one_score = calculate_one_score(score)
# print("%.2f" % one_score)
# print(score.neg_score())

# with open("pos_sentiments.txt", "w") as negs:
#     sentiments = []
#     for sentence in jsonSentences:
#         sents = []
#         for word in sentence:
#             try:
#                 score = get_score(word["word"], word["posTag"])
#                 sents.append(score)
#                 negs.writelines(str(score))
#                 print(score)
#             except:
#                 e = sys.exc_info()[0]
#                 print(e)
#         sentiments.append(sents)
#         negs.write("\n")

sentiments = []
for sentence in jsonSentences:
    total = 0.0
    for word in sentence:
        scores = get_score(word["word"], word["posTag"])
        score = calculate_one_score(scores)
#         print("Score for {0}: {1}".format(word["word"], score))
        total += score
    sentiments.append(total)
#     print("Final score for sentence {0} is --> {1}".format(sentence, total))
print(sentiments)

[0.2169160583941606, 0.1752596784799625, -0.16194584202436468, 0.05557554320297834, 0.20177769281222752, -0.37568636012541945, 0.21441605839416059, -0.028745072273324575, 0.10714288528601175, -0.17658248545051994]


In [12]:
def get_score(word, postag):
    scores = []
    counter = 1
    true_score = 0
    while True:
        try:
            if postag == "Noun":  # NOUN
                score = swn.senti_synset(word + '.n.0' + str(counter))
            elif postag == "Verb":  # VERB
                score = swn.senti_synset(word + '.v.0' + str(counter))
            elif postag == "Adjective":  # ADJECTIVE
                score = swn.senti_synset(word + '.a.0' + str(counter))
            elif postag == "Adverb":  # ADVERB
                score = swn.senti_synset(word + '.r.0' + str(counter))
            
#             print("POSITIVE: " + str(score.pos_score()))
#             print("NEGATIVE: " + str(score.neg_score()))
            true_score = score.pos_score() - score.neg_score()
#             print("TRUE: " + str(true_score))
            scores.append(true_score)
            counter += 1
        except:
#             print ("Unexpected error:", sys.exc_info()[0])
            break
            
    return scores

In [27]:
def calculate_one_score(scores):
    if len(scores) == 0:
        return 0.0
    
    numerator = 0.0
    denominator = 0.0
    for i in range(len(scores)):
        numerator += scores[i] * (1 / (i+2))
        denominator += 1 / (i+1)
    return numerator / denominator
    